# **Лабораторная работа №3**

Diana Kim ADA-2403M

## **Подключение GPU и CUDA**

In [1]:
!nvidia-smi

Mon Dec 29 19:54:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


## **Практические задания**

создаю файл main.cu прямо в Colab

In [3]:
%%writefile main.cu
#include <cuda_runtime.h>               // для функций CUDA, которые мне надо
#include <device_launch_parameters.h>                 // подключает переменные threadIdx, blockIdx и параметры запуска ядра
#include <iostream>                   // ввод/вывод
#include <vector>                            // подключает контейнер vector для динамических массивов
#include <random>                        // для генерации случайных чисел
#include <chrono>                                 // для измерения времени выполнения
#include <algorithm>                 // для алгоритмов, которые мне надо

#define INF 2147483647               // используется как заполнитель для хвоста массива
#define CUDA_CHECK(call) do {                                                     /* объявляет макрос для проверки ошибок */ \
    cudaError_t err = (call);                                                     /* вызывает CUDA-функцию и сохраняет код ошибки */ \
    if (err != cudaSuccess) {                                                     /* проверяет, что вызов завершился успешно */ \
        std::cerr << "CUDA error: " << cudaGetErrorString(err)                    /* выводит текст ошибки */ \
                  << " at " << __FILE__ << ":" << __LINE__ << "\n";               /* выводит файл и строку */ \
        std::exit(1);                                                             /* завершает программу с ошибкой */ \
    }                                                                             /* закрывает if */ \
} while(0)                                                                        /* делает макрос одной командой */

static const int CHUNK = 256;                // задает размер подмассива на один блок



// CPU Merge sort, ЗАДАНИЕ 4
static void cpuMergeSort(std::vector<int>& a) {            // объявляет функцию последовательной сортировки слиянием на CPU
    int n = (int)a.size();                                       // получает размер массива и приводит к int
    std::vector<int> tmp(n);              // создаёт временный массив tmp такого же размера
    for (int width = 1; width < n; width *= 2) {               // увеличивает размер отсортированных блоков
        for (int i = 0; i < n; i += 2 * width) {                  // проходит по массиву парами блоков длины width
            int l = i;                         // запоминает левую границу первой части
            int m = std::min(i + width, n);                 // вычисляет границу между частями при этом не выходя за n
            int r = std::min(i + 2 * width, n);                       // вычисляет правую границу второй части при этом не выходя за n
            int p = l, q = m, k = l;                     // создает указатели p по левой части, q по правой, k куда писать
            while (p < m && q < r) tmp[k++] = (a[p] <= a[q]) ? a[p++] : a[q++];         // сливает две части, берёт меньший элемент и записывает в tmp
            while (p < m) tmp[k++] = a[p++];                     // дописывает остаток левой части, если он остался
            while (q < r) tmp[k++] = a[q++];                     // дописывает остаток правой части, если он остался
        }
        a.swap(tmp);                  // меняет местами a и tmp
    }
}



// CPU Quick sort, ЗАДАНИЕ 4
static void cpuQuickSort(std::vector<int>& a) {                  // объявляет функцию быстрой сортировки на CPU
    std::sort(a.begin(), a.end());           // сортирует vector стандартной функцией
}


// CPU Heap sort, ЗАДАНИЕ 4
static void cpuHeapSort(std::vector<int>& a) {                     // объявляет функцию пирамидальной сортировки на CPU
    std::make_heap(a.begin(), a.end());                       // превращает массив в max heap
    std::sort_heap(a.begin(), a.end());                       // извлекает элементы из кучи и получает отсортированный массив
}


// ЗАДАНИЕ 1
__global__ void mergePass(const int* input, int* output, int n, int width) {           // объявляет CUDA ядро для слияния двух отсортированных частей
    int pairIdx = blockIdx.x;                       // получает номер пары, которую обрабатывает блок
    int start = pairIdx * (2 * width);                            // вычисляет стартовый индекс пары прогонов
    if (start >= n) return;                               // выходит, если блок вышел за пределы массива

    int mid = (start + width < n) ? (start + width) : n;                // вычисляет середину (конец первого прогона), ограничивая n
    int end = (start + 2 * width < n) ? (start + 2 * width) : n;              // вычисляет конец второго прогона, ограничивая n

    if (threadIdx.x == 0) {                 // заставляет выполнять слияние только нулевой поток блока
        int i = start, j = mid, k = start;              // задает индексы, i по левому прогону, j по правому, k позиция записи
        while (i < mid && j < end) {           // выполняет слияние пока в обоих прогонах есть элементы
            if (input[i] <= input[j]) output[k++] = input[i++];              // если левый элемент меньше или равен, то записывает его
            else                      output[k++] = input[j++];              // иначе записывает правый элемент
        }
        while (i < mid) output[k++] = input[i++];                 // дописывает остаток левого прогона
        while (j < end) output[k++] = input[j++];               // дописывает остаток правого прогона
    }
}


// GPU Merge sort, ЗАДАНИЕ 1
__global__ void sortChunksBitonic(const int* in, int* out, int n) {             // объявляет CUDA ядро где один блок сортирует один CHUNK
    __shared__ int s[CHUNK];             // создает shared массив внутри блока для быстрой сортировки
    int tid  = threadIdx.x;                          // получает номер потока внутри блока
    int base = blockIdx.x * CHUNK;             // вычисляет начало куска для этого блока
    int idx  = base + tid;                 // вычисляет глобальный индекс элемента массива
    s[tid] = (idx < n) ? in[idx] : INF;            // записывает элемент в shared или INF, если idx вышел за n
    __syncthreads();                     // синхронизирует потоки, чтобы shared заполнен полностью
    for (int k = 2; k <= CHUNK; k <<= 1) {              // увеличивает размер битонного блока в 2 раза
        for (int j = k >> 1; j > 0; j >>= 1) {                 // задает расстояние для сравнения элементов
            int ixj = tid ^ j;                       // вычисляет индекс партнера для сравнения через XOR
            if (ixj > tid) {                       // проверяет условие, чтобы сравнение выполнялось один раз на пару
                bool asc = ((tid & k) == 0);                   // определяет направление сортировки, вверх или вниз
                int a = s[tid], b = s[ixj];             // берет два сравниваемых значения из shared
                if (asc) { if (a > b) { s[tid] = b; s[ixj] = a; } }               // если направление вверх и a>b, то меняет местами
                else     { if (a < b) { s[tid] = b; s[ixj] = a; } }             // если направление вниз и a<b, то меняет местами
            }
            __syncthreads();             // синхронизирует потоки после каждого шага сравнения
        }
    }
    if (idx < n) out[idx] = s[tid];                     // записывает отсортированное значение обратно в глобальную память
}



// GPU Quick sort, ЗАДАНИЕ 2
__device__ void insertionSort(int* a, int n) {                   // объявляет функцию сортировки вставками для device
    for (int i = 1; i < n; i++) {                         // проходит по элементам начиная со второго
        int key = a[i];                      // сохраняет текущий элемент как key
        int j = i - 1;                      // устанавливает j на элемент слева от key
        while (j >= 0 && a[j] > key) {                // двигает элементы вправо, пока они больше key
            a[j + 1] = a[j];                     // сдвигает элемент вправо на одну позицию
            j--;                // уменьшает индекс j
        }
        a[j + 1] = key;           // вставляет key на правильное место
    }
}

__global__ void sortChunksQuickSimple(const int* in, int* out, int n) {         // объявляет CUDA ядро для quick варианта, ЗАДАНИЕ 2
    __shared__ int s[CHUNK];                  // создает shared массив для куска данных
    int tid  = threadIdx.x;                        // получает номер потока в блоке
    int base = blockIdx.x * CHUNK;                // вычисляет начало куска для этого блока
    int idx  = base + tid;                            // вычисляет глобальный индекс элемента
    s[tid] = (idx < n) ? in[idx] : INF;            // записывает элемент в shared или INF, если вышел за n
    __syncthreads();               // синхронизирует потоки после загрузки
    if (tid == 0) insertionSort(s, CHUNK);                         // запускает сортировку вставками одним потоком
    __syncthreads();                           // синхронизирует потоки после сортировки
    if (idx < n) out[idx] = s[tid];                               // записывает отсортированный кусок обратно в out
}




// GPU Heap sort, ЗАДАНИЕ 3
__device__ void siftDown(int* a, int start, int end) {                 // объявляет функцию просеивания вниз для кучи
    int root = start;                       // задает root как начальный индекс
    while (true) {                  // запускает бесконечный цикл
        int child = 2 * root + 1;                            // вычисляет индекс левого ребенка
        if (child >= end) return;                     // выходит за груницу кучи, если детей нет
        int swapIdx = root;                     // задает swapIdx как root, кандидат на обмен
        if (a[swapIdx] < a[child]) swapIdx = child;                  // выбирает большего из root и левого ребенка
        if (child + 1 < end && a[swapIdx] < a[child + 1]) swapIdx = child + 1;             // сравнивает еще и с правым ребенком

        if (swapIdx == root) return;                       // выходит, если обмен не нужен
        int t = a[root]; a[root] = a[swapIdx]; a[swapIdx] = t;            // меняет местами root и выбранного ребенка
        root = swapIdx;                 // продолжает просеивание с новой позиции root
    }
}

__device__ void heapSortChunk(int* a, int n) {                       // объявляет heapsort для одного чанка, ЗАДАНИЕ 3
    for (int i = n / 2 - 1; i >= 0; --i) siftDown(a, i, n);               // строит max heap, просеивая все внутренние узлы
    for (int end = n - 1; end > 0; --end) {                                     // извлекает максимум, уменьшая размер кучи
        int t = a[0]; a[0] = a[end]; a[end] = t;                    // меняет местами первый и последний элемент кучи
        siftDown(a, 0, end);                                   // восстанавливает свойство кучи для уменьшенной кучи
    }
}

__global__ void sortChunksHeapSimple(const int* in, int* out, int n) {          // объявляет CUDA ядро heapsort на чанках, ЗАДАНИЕ 3
    __shared__ int s[CHUNK];                  // создаёт shared массив для данных блока
    int tid  = threadIdx.x;                            // получает номер потока в блоке
    int base = blockIdx.x * CHUNK;                      // вычисляет начало куска
    int idx  = base + tid;                           // вычисляет глобальный индекс
    s[tid] = (idx < n) ? in[idx] : INF;                         // загружает данные в shared или INF, если вышел за n
    __syncthreads();                             // синхронизирует потоки после загрузки
    if (tid == 0) heapSortChunk(s, CHUNK);                // запускает heapsort одним потоком для простоты
    __syncthreads();                                              // синхронизирует потоки после сортировки
    if (idx < n) out[idx] = s[tid];                     // записывает отсортированные данные обратно в out
}



// GPU pipeline
static float gpuSortPipeline(int* d_in, int* d_tmp, int n, int mode) {               // объявляет функцию, которая запускает нужную GPU сортировку
    // mode: 0=merge, 1=quick, 2=heap                           // поясняет значения параметра mode
    cudaEvent_t st, en;                             // объявляет CUDA-события для измерения времени
    CUDA_CHECK(cudaEventCreate(&st));                        // создает событие начала
    CUDA_CHECK(cudaEventCreate(&en));                       // создает событие конца
    CUDA_CHECK(cudaEventRecord(st));                        // записывает событие старта таймера
    int blocks = (n + CHUNK - 1) / CHUNK;                      // вычисляет количество блоков для обработки всех элементов
    if (mode == 0)      sortChunksBitonic<<<blocks, CHUNK>>>(d_in, d_tmp, n);               // запускает сортировку чанков битоником
    else if (mode == 1) sortChunksQuickSimple<<<blocks, CHUNK>>>(d_in, d_tmp, n);                         // запускает сортировку чанков quick вариантом
    else               sortChunksHeapSimple<<<blocks, CHUNK>>>(d_in, d_tmp, n);                         // запускает сортировку чанков heapsort вариантом
    CUDA_CHECK(cudaGetLastError());                           // проверяет ошибки запуска CUDA-ядра
    CUDA_CHECK(cudaDeviceSynchronize());                               // ждет завершения сортировки чанков
    int *src = d_tmp, *dst = d_in;                        // задает указатели src источник, dst приемник

    for (int width = CHUNK; width < n; width *= 2) {                        // увеличивает размер отсортированных прогонов в 2 раза
        int pairs = (n + (2 * width) - 1) / (2 * width);                       // вычисляет количество пар для слияния на этом шаге
        mergePass<<<pairs, 256>>>(src, dst, n, width);                // запускает слияние пар прогонов
        CUDA_CHECK(cudaGetLastError());                              // проверяет ошибки запуска mergePass
        CUDA_CHECK(cudaDeviceSynchronize());                          // ждет завершения mergePass
        std::swap(src, dst);               // меняет местами src и dst для следующего шага
    }

    if (src != d_in) {                    // проверяет, что итог лежит не в d_in
        CUDA_CHECK(cudaMemcpy(d_in, src, n * sizeof(int), cudaMemcpyDeviceToDevice));             // копирует итог в d_in, если нужно
    }

    CUDA_CHECK(cudaEventRecord(en));                       // записывает событие окончания таймера
    CUDA_CHECK(cudaEventSynchronize(en));              // ждет завершения события en
    float ms = 0.f;                             // создает переменную для времени в миллисекундах
    CUDA_CHECK(cudaEventElapsedTime(&ms, st, en));                    // вычисляет время между st и en
    CUDA_CHECK(cudaEventDestroy(st));                    // удаляет событие st
    CUDA_CHECK(cudaEventDestroy(en));                       // удаляет событие en
    return ms;                               // возвращает измеренное время GPU сортировки
}

static float gpuMergeSort(int* d_in, int* d_tmp, int n) { return gpuSortPipeline(d_in, d_tmp, n, 0); }              // вызывает GPU merge sort через общий конвейер, ЗАДАНИЕ 1
static float gpuQuickSort(int* d_in, int* d_tmp, int n) { return gpuSortPipeline(d_in, d_tmp, n, 1); }                  // вызывает GPU quick sort через общий конвейер, ЗАДАНИЕ 2
static float gpuHeapSort (int* d_in, int* d_tmp, int n) { return gpuSortPipeline(d_in, d_tmp, n, 2); }              // вызывает GPU heap sort через общий конвейер, ЗАДАНИЕ 3



static double timeCpu(void(*fn)(std::vector<int>&), const std::vector<int>& input) {             // объявляет функцию измерения времени CPU сортировки, ЗАДАНИЕ 4
    std::vector<int> a = input;                                      // копирует входной массив, чтобы не портить оригинал
    auto t0 = std::chrono::high_resolution_clock::now();                // запоминает время начала
    fn(a);                                    // вызывает переданную CPU сортировку
    auto t1 = std::chrono::high_resolution_clock::now();                 // запоминает время конца
    if (!std::is_sorted(a.begin(), a.end())) {                       // проверяет, что массив действительно отсортирован
        std::cerr << "CPU sort FAILED\n";                       // выводит сообщение об ошибке
        std::exit(1);                                // завершает программу с ошибкой
    }
    std::chrono::duration<double, std::milli> ms = t1 - t0;                   // вычисляет длительность в миллисекундах
    return ms.count();                                             // возвращает время CPU сортировки
}
static float timeGpu(float(*fn)(int*, int*, int), const std::vector<int>& input) {               // объявляет функцию измерения времени GPU сортировки, ЗАДАНИЕ 4
    int n = (int)input.size();                                  // получает размер массива и приводит к int
    int *d_in = nullptr, *d_tmp = nullptr;                             // объявляет указатели на память GPU
    CUDA_CHECK(cudaMalloc(&d_in,  n * sizeof(int)));                           // выделяет память на GPU для входного массива
    CUDA_CHECK(cudaMalloc(&d_tmp, n * sizeof(int)));                          // выделяет память на GPU для временного массива
    CUDA_CHECK(cudaMemcpy(d_in, input.data(), n * sizeof(int), cudaMemcpyHostToDevice));         // копирует данные с CPU на GPU
    float ms = fn(d_in, d_tmp, n);                   // запускает выбранную GPU сортировку и получает время
    std::vector<int> out(n);                                        // создает массив out для результата на CPU
    CUDA_CHECK(cudaMemcpy(out.data(), d_in, n * sizeof(int), cudaMemcpyDeviceToHost));           // копирует результат с GPU обратно на CPU
    if (!std::is_sorted(out.begin(), out.end())) {                // проверяет, что результат отсортирован
        std::cerr << "GPU sort FAILED\n";                       // выводит сообщение об ошибке
        std::exit(1);                        // завершает программу с ошибкой
    }
    CUDA_CHECK(cudaFree(d_in));                    // освобождает память d_in на GPU
    CUDA_CHECK(cudaFree(d_tmp));            // освобождает память d_tmp на GPU
    return ms;                               // возвращает время GPU сортировки
}

static void runN(int n) {                // объявляет функцию запуска теста для конкретного размера n, ЗАДАНИЕ 4
    std::mt19937 rng(123);                      // создает генератор случайных чисел с фиксированным seed
    std::uniform_int_distribution<int> dist(0, 1000000);        // задает диапазон случайных чисел

    std::vector<int> a(n);                  // создает массив a размера n
    for (int i = 0; i < n; i++) a[i] = dist(rng);                     // заполняет массив случайными числами

    std::cout << "\n N = " << n << " \n" << std::flush;               // выводит заголовок теста и сразу сбрасывает буфер

    std::cout << "CPU merge: " << timeCpu(cpuMergeSort, a) << " ms\n" << std::flush;       // измеряет и выводит время CPU merge sort
    std::cout << "CPU quick: " << timeCpu(cpuQuickSort, a) << " ms\n" << std::flush;             // измеряет и выводит время CPU quick sort
    std::cout << "CPU heap : " << timeCpu(cpuHeapSort,  a) << " ms\n" << std::flush;           // измеряет и выводит время CPU heap sort

    CUDA_CHECK(cudaFree(0));        // выполняет прогрев GPU

    std::cout << "GPU merge: " << timeGpu(gpuMergeSort, a) << " ms\n" << std::flush;          // измеряет и выводит время GPU merge sort
    std::cout << "GPU quick: " << timeGpu(gpuQuickSort, a) << " ms\n" << std::flush;            // измеряет и выводит время GPU quick sort
    std::cout << "GPU heap : " << timeGpu(gpuHeapSort,  a) << " ms\n" << std::flush;         // измеряет и выводит время GPU heap sort
}


int main() {             // ЗАДАНИЕ 4
    runN(10000);                            // запускает тест для массива из 10000 элементов
    runN(100000);                      // запускает тест для массива из 100000 элементов
    runN(1000000);                       // запускает тест для массива из 1000000 элементов
    return 0;             // завершает программу успешно
}

Writing main.cu


In [4]:
!rm -f app
!nvcc main.cu -O2 -o app -gencode arch=compute_75,code=sm_75
!./app


 N = 10000 
CPU merge: 1.69602 ms
CPU quick: 0.601974 ms
CPU heap : 1.08445 ms
GPU merge: 5.31507 ms
GPU quick: 7.2287 ms
GPU heap : 5.57891 ms

 N = 100000 
CPU merge: 11.6598 ms
CPU quick: 7.34017 ms
CPU heap : 13.5176 ms
GPU merge: 44.6602 ms
GPU quick: 50.5612 ms
GPU heap : 45.9671 ms

 N = 1000000 
CPU merge: 146.489 ms
CPU quick: 94.6492 ms
CPU heap : 179.177 ms
GPU merge: 231.543 ms
GPU quick: 190.517 ms
GPU heap : 176.507 ms
